[kaggle-house-price](../chapter_multilayer-perceptrons/kaggle-house-price.ipynb)

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview

In [10]:
import hashlib
import os
import tarfile
import zipfile
import requests

DATA_HUB = dict()
DATA_URL = "http://d2l-data.s3-accelerate.amazonaws.com/"


def download(name, cache_dir=os.path.join("..", "data")):
    """下载一个DATA_HUB中的文件，返回本地文件名"""
    assert name in DATA_HUB, f"{name} 不存在于{DATA_HUB}"
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split("/")[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, "rb") as f:
            while True:
                data = f.read(1048576)
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname  # 命中缓存(本地文件)
    print(f"正在从{url}下载{fname}...")
    r = requests.get(url, stream=True, verify=True)
    with open(fname, "wb") as f:
        f.write(r.content)
    return fname


def download_extract(name, folder=None):
    """下载并解压zip/tar文件"""
    fname = download(name)
    base_dir = os.path.dirname(fname)
    data_dir, ext = os.path.splitext(fname)
    if ext == ".zip":
        fp = zipfile.ZipFile(fname, "r")
    elif ext in (".tar", ".gz"):
        fp = tarfile.open(fname, "r")
    else:
        assert False, "只有zip/tar文件可以被解压缩"
    fp.extractall(base_dir)
    return os.path.join(base_dir, folder) if folder else data_dir  # folder不填返回data_dir，


def download_all():
    """下载DATA_HUB中得所有文件"""
    for name in DATA_HUB:
        download(name)




In [11]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

DATA_HUB["kaggle_house_train"] = (
    DATA_URL + "kaggle_house_pred_train.csv",
    "585e9cc93e70b39160e7921475f9bcd7d31219ce",
)
DATA_HUB["kaggle_house_test"] = (  # @save
    DATA_URL + "kaggle_house_pred_test.csv",
    "fa19780a7b011d9b009e8bff8e99922a8ee2eb90",
)


train_data = pd.read_csv(download("kaggle_house_train"))
test_data = pd.read_csv(download("kaggle_house_test"))

all_features = pd.concat(
    (train_data.iloc[:, 1:-1], test_data.iloc[:, 1:])
)  # 都去掉id列，train去掉label列(SalePrice)


# 数值有大有小，让每一列均值为0，方差为1
# all_features是 pandas.core.frame.DataFrame类, all_features.dtypes返回每列的类型 ; .index返回对应的列名
numeric_features = all_features.dtypes[
    all_features.dtypes != "object"
].index  # 假设不是object就是数值
# apply默认按axis=0(列操作)
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean())
    / (x.std())  # x是一列数据 shape:(2919,)
    # 若无法获得测试数据，则可根据训练数据计算均值和标准差
)
# 在标准化数据之后，所有均值(均值为0)消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

# 处理离散值 用one hot替换
# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=True)  # shape:(2919, 331)

# 最后，通过`values`属性，我们可以 从`pandas`格式中提取NumPy格式，并将其转换为张量表示用于训练。
n_train = train_data.shape[0]  # 1460 行
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32)
train_labels = torch.tensor(
    train_data.SalePrice.values.reshape(-1, 1),
    dtype=torch.float32,  # train_data.SalePrice.values array
)



使用特征缩放的作用是（标准化，归一化）：
使不同量纲的特征处于同一数值量级，减少方差大的特征的影响，使模型更准确。
加快学习算法的收敛速度。

In [12]:
loss = nn.MSELoss()
in_features = train_features.shape[1]  # 列数 331


def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1))
    return net


def log_rmse(net, features, labels):
    """均方根误差"""
    # https://www.jianshu.com/p/9ee85fdad150
    # 比较大的，正的值用这个。
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    # torch.clamp(input, min, max) 将input张量里的每个值限制在 [min,max] 内
    clipped_preds = torch.clamp(net(features), 1, float("inf"))
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))
    return rmse.item()


def train(
    net,
    train_features,
    train_labels,
    test_features,
    test_labels,
    num_epochs,
    learning_rate,
    weight_decay,
    batch_size,
):
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    # 这里使用的是Adam优化算法
    optimizer = torch.optim.Adam(
        net.parameters(), lr=learning_rate, weight_decay=weight_decay
    )
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        # 训练完一个epoch后，记录损失数据
        train_ls.append(log_rmse(net, train_features, train_labels))  # 训练均方根损失
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))  # 测试均方根损失
    return train_ls, test_ls


def get_k_fold_data(k, i, X, y):
    """获取k折交叉验证数据集,i:第几折为验证集"""
    assert k > 1
    fold_size = X.shape[0] // k  # //代表整除 丢弃分数. 每折的大小
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(
            j * fold_size, (j + 1) * fold_size
        )  # 返回一个切片对象  slice(start, stop[, step])  idx就是第k折的开始到结尾的切片;这里是否会超过数据大小？
        X_part, y_part = X[idx, :], y[idx]  # X要第K折的所有列
        if j == i:
            # 到验证集这一折的时候
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            # 第一次设置训练集的时候
            X_train, y_train = X_part, y_part
        else:
            # 添加训练集的时候
            X_train = torch.cat([X_train, X_part], 0)  # 按维度0拼接
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid



2.5